In [8]:
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path

# Load environment variables (expects your API key in .env)
load_dotenv()

openai = OpenAI()

# Read the transcript file
input_file = Path("data/combined_transcript.txt")
transcript_text = input_file.read_text(encoding="utf-8")

# Call the model
response = openai.chat.completions.create(
    model='openai.gpt-4o-mini',
    messages=[
        {'role': 'system', 'content': 'You are a graduate assistant that reviews lecture notes from the INFO 5940 class at Cornell. Please step through the entire set of notes given in the provided transcipt file and format the content into easily readable sections, always creating an actions section and an overview at the top, followed by detailed notes, taking care not to omit any information from the transcript and to never add any factual information not in the transcript. When you are done, think about your notes to determine if they are logical and would make sense to a student who did not attend the lecture and if you have the 3 sections: overview, actions, and detailed notes. '},
        {'role': 'user', 'content': transcript_text},
    ],
).choices[0].message.content

# Save reformatted transcript
output_file = Path("data/combined_transcript_reformatted.txt")
output_file.write_text(response, encoding="utf-8")

print(f'GPT-4o-mini:\n\n{response}\n\n')
print(f'Reformatted transcript saved to {output_file.resolve()}')

GPT-4o-mini:

### Overview
In this lecture of INFO 5940 at Cornell, various topics surrounding Artificial Intelligence (AI) were discussed. The class began with student experiences related to AI applications, specifically focusing on tools like ChatGPT and Gemini. Discussions included the implications of AI in design, job searching, and the inner critic phenomena. The lecturer emphasized the importance of understanding AI technology's potential and its associated challenges, particularly in the context of developing apps using AI models. A segment was dedicated to demonstrating the OpenAI SDK, discussing its functionalities, and explaining how API usage differs from general AI models like ChatGPT.

### Actions
1. Share experiences with AI applications, including successes and challenges.
2. Familiarize yourself with Gemini, ChatGPT, and the implications of AI in your personal and professional life.
3. Ensure participation in upcoming Zoom sessions and prepare for the development enviro